In [1]:
%load_ext autoreload
%autoreload 2

# SequencePointcloudRegistrationPipeline

A module that implements an algorithm for optimizing the position and orientation of a vehicle in space based on a sequence of multimodal data using neural network methods.

In [2]:
from time import time

import faiss

from tqdm import tqdm

from hydra.utils import instantiate
import numpy as np
from omegaconf import OmegaConf
from scipy.spatial.transform import Rotation
import torch
from torch.utils.data import DataLoader


try:
    from geotransformer.utils.registration import compute_registration_error
    from geotransformer.utils.pointcloud import get_transform_from_rotation_translation
except ImportError:
    print("WARNIGN: geotransformer not installed, registration error will not be computed")

from opr.datasets.itlp import ITLPCampus
from opr.pipelines.place_recognition import PlaceRecognitionPipeline
from opr.pipelines.registration.pointcloud import SequencePointcloudRegistrationPipeline

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
DATABASE_TRACK_DIR = "/home/docker_opr/Datasets/ITLP-Campus-data/subsampled_data/indoor/00_2023-10-25-night"
QUERY_TRACK_DIR = "/home/docker_opr/Datasets/ITLP-Campus-data/subsampled_data/indoor/01_2023-11-09-twilight"

REGISTRATION_MODEL_CONFIG_PATH = "../../configs/model/registration/geotransformer_kitti.yaml"
REGISTRATION_WEIGHTS_PATH = "../../weights/registration/geotransformer_kitti.pth"

SENSOR_SUITE = ["front_cam", "back_cam", "lidar"]

BATCH_SIZE = 64
NUM_WORKERS = 4
DEVICE = "cuda"

MODEL_CONFIG_PATH = "../configs/model/place_recognition/multi-image_lidar_late-fusion.yaml"
WEIGHTS_PATH = "../weights/place_recognition/multi-image_lidar_late-fusion_nclt.pth"

In [4]:
def pose_to_matrix(pose):
    """From the 6D poses in the [tx ty tz qx qy qz qw] format to 4x4 pose matrices."""
    position = pose[:3]
    orientation_quat = pose[3:]
    rotation = Rotation.from_quat(orientation_quat)
    pose_matrix = np.eye(4)
    pose_matrix[:3,:3] = rotation.as_matrix()
    pose_matrix[:3,3] = position
    return pose_matrix


def compute_error(estimated_pose, gt_pose):
    """For the 6D poses in the [tx ty tz qx qy qz qw] format."""
    estimated_pose = pose_to_matrix(estimated_pose)
    gt_pose = pose_to_matrix(gt_pose)
    error_pose = np.linalg.inv(estimated_pose) @ gt_pose
    dist_error = np.sum(error_pose[:3, 3]**2) ** 0.5
    r = Rotation.from_matrix(error_pose[:3, :3])
    rotvec = r.as_rotvec()
    angle_error = (np.sum(rotvec**2)**0.5) * 180 / np.pi
    angle_error = abs(90 - abs(angle_error-90))
    return dist_error, angle_error

def compute_translation_error(gt_pose, pred_pose):
    """For the 4x4 pose matrices."""
    gt_trans = gt_pose[:3, 3]
    pred_trans = pred_pose[:3, 3]
    error = np.linalg.norm(gt_trans - pred_trans)
    return error

def compute_rotation_error(gt_pose, pred_pose):
    """For the 4x4 pose matrices."""
    gt_rot = Rotation.from_matrix(gt_pose[:3, :3])
    pred_rot = Rotation.from_matrix(pred_pose[:3, :3])
    error = Rotation.inv(gt_rot) * pred_rot
    error = error.as_euler('xyz', degrees=True)
    error = np.linalg.norm(error)
    return error

def compute_absolute_pose_error(gt_pose, pred_pose):
    """For the 4x4 pose matrices."""
    rotation_error = compute_rotation_error(gt_pose, pred_pose)
    translation_error = compute_translation_error(gt_pose, pred_pose)
    return rotation_error, translation_error

In [5]:
db_dataset = ITLPCampus(
    dataset_root=DATABASE_TRACK_DIR,
    sensors=["front_cam", "back_cam", "lidar"],
    mink_quantization_size=0.5,
)

In [6]:
db_dataloader = DataLoader(
    db_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    collate_fn=db_dataset.collate_fn,
)


In [7]:
model_config = OmegaConf.load(MODEL_CONFIG_PATH)
model = instantiate(model_config)
model.load_state_dict(torch.load(WEIGHTS_PATH))
model = model.to(DEVICE)
model.eval();

In [8]:
geotransformer = instantiate(OmegaConf.load(REGISTRATION_MODEL_CONFIG_PATH))

registration_pipe = SequencePointcloudRegistrationPipeline(
    model=geotransformer,
    model_weights_path=REGISTRATION_WEIGHTS_PATH,
    device="cuda",  # the GeoTransformer currently only supports CUDA
    voxel_downsample_size=0.5,  # recommended for geotransformer_kitti configuration
)

In [9]:
pipe = PlaceRecognitionPipeline(
    database_dir=DATABASE_TRACK_DIR,
    model=model,
    model_weights_path=WEIGHTS_PATH,
    device=DEVICE,
)


In [10]:
query_dataset = ITLPCampus(
    dataset_root=QUERY_TRACK_DIR,
    sensors=SENSOR_SUITE,
    mink_quantization_size=0.5,
)


In [11]:
import gc

PR_THRESHOLD = 25.0

db_matches = {}

pr_matches = []
dist_errors = []
angle_errors = []
times = []

for i in tqdm(range(1, len(query_dataset)), total=len(query_dataset)-1):
    query = query_dataset[i]
    output_idx = pipe.infer(query)["idx"]
    db_matches[i] = output_idx

del pipe
gc.collect()
torch.cuda.empty_cache()


  0%|          | 0/1309 [00:00<?, ?it/s]

100%|██████████| 1309/1309 [00:51<00:00, 25.42it/s]


In [12]:
query_dataset = ITLPCampus(
    dataset_root=QUERY_TRACK_DIR,
    sensors=SENSOR_SUITE,
    mink_quantization_size=0.5,
    max_point_distance=20,
)

db_dataset = ITLPCampus(
    dataset_root=DATABASE_TRACK_DIR,
    sensors=SENSOR_SUITE,
    mink_quantization_size=0.5,
    max_point_distance=20,
)

In [ ]:
for i in tqdm(range(1, len(query_dataset)), total=len(query_dataset)-1):
    query = query_dataset[i]
    query_seq = [query_dataset[i-1]["pointcloud_lidar_coords"], query_dataset[i]["pointcloud_lidar_coords"]]
    output_idx = db_matches[i]
    db_match = db_dataset[output_idx]
    torch.cuda.empty_cache()
    db_pose = pose_to_matrix(db_match["pose"])
    db_pc = db_match["pointcloud_lidar_coords"]
    t = time()
    estimated_transformation = registration_pipe.infer(query_seq, db_pc)
    times.append(time() - t)
    optimized_pose = db_pose @ estimated_transformation
    torch.cuda.empty_cache()
    angle_error, dist_error = compute_absolute_pose_error(optimized_pose, pose_to_matrix(query["pose"]))
    pr_matches.append(dist_error < PR_THRESHOLD)
    dist_errors.append(dist_error)
    angle_errors.append(angle_error)

times = times[1:]  # the first query is always slower

In [5]:
print(f"Recall@1: {(np.mean(pr_matches))*100:.2f}")
print(f"Mean distance error: {np.mean(dist_errors):.2f}, mean angle error: {np.mean(angle_errors):.2f}")
print(f"Median distance error: {np.median(dist_errors):.2f}, median angle error: {np.median(angle_errors):.2f}")

Recall@1: 68.63
Mean distance error: 47.38, mean angle error: 5.54
Median distance error: 1.30, median angle error: 2.84


In [ ]:
print(f"Mean inference time: {np.mean(times)*1000:.2f} ms, median inference time: {np.median(times)*1000:.2f} ms")